## **Classificador de texto SMS baseado em rede neural**

Carregando Bibliotecas

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
  # import libraries
  try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
  except Exception:
    pass
  import tensorflow as tf
  import pandas as pd
  from tensorflow import keras
  !pip install tensorflow-datasets
  import tensorflow_datasets as tfds
  import numpy as np
  import matplotlib.pyplot as plt


print(tf.__version__)

Carregando dados

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

Leitura dos dados e separação do texto com o rotulo

In [ ]:
with open('/content/train-data.tsv') as arquivo:
  leitura = arquivo.readlines()

texto_treino = []
rotulo_treino = []

for linha in leitura:
  texto_treino.append(linha.split(sep = '\t')[1])
  rotulo_treino.append(linha.split(sep = '\t')[0])

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000) #Transformado o texto em uma sequencia de numeros
tokenizer.fit_on_texts(texto_treino) #Constroi o vocabulário com base nas palavras presentes nos textos
sequences = tokenizer.texts_to_sequences(texto_treino) #Converte os texto em sequencias
X_train = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=100) #Preenchimento da sequencia para que todos tenham o mesmo comprimento
y_train = np.array([1 if label == 'spam' else 0 for label in rotulo_treino], dtype=int) #Criação de array numpy, atribuindo 1 para 'spam' e 0 para 'ham'

Crianção do modelo

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64),  # Remova input_length
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Treinamento do modelo

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Criação da função para processar a mensagem

In [ ]:
def predict_message(pred_text):

    pred_seq = tokenizer.texts_to_sequences([pred_text])
    pred_seq = tf.keras.preprocessing.sequence.pad_sequences(pred_seq, maxlen=100)

    # Previsão
    prediction = model.predict(pred_seq)[0, 0]

    # Classificar como 'ham' se a previsão for inferior a 0.5, senão, como 'spam'
    return 'ham' if prediction < 0.5 else 'spam'

Avaliação

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
